In [1]:
#Purpose:
#Show rules working (Budget, MaxFromTeam)
#Show violations + fixing them
#Show advisor recommendations
#Show generator + filters

In [2]:
#imports
from pathlib import Path
import sys
from collections import Counter

ROOT = Path.cwd().parent
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

from src.data_api import fetch_bootstrap_static
from src.transform import build_player_objects
from src.models import Squad
from src.rules import BudgetRule, FormationRule, MaxFromTeamRule, RuleViolation
from src.advisor import recommend_players
from src.advisor_engine import Advisor

In [3]:
#load players and build a test squad
data = fetch_bootstrap_static()
player_objects = build_player_objects(data)

squad = Squad(budget=20.0)  # small budget to trigger rule
for p in player_objects[:4]:
    squad.add_player(p)

print("Test squad:", squad)

Test squad: Squad(players=4, cost=18.0/20.0)


In [8]:
#test rules
rules = [
    BudgetRule(),
    MaxFromTeamRule(max_per_team=3),
]

for r in rules:
    try:
        r.validate(squad)
        print(f"{r.__class__.__name__}: OK")
    except RuleViolation as e:
        print(f"{r.__class__.__name__}: VIOLATION -> {e}")

BudgetRule: OK
MaxFromTeamRule: VIOLATION -> Too many players from team 1: 4 > 3


In [5]:
#advisor and recommendations tester
advisor = Advisor(rules=[BudgetRule(), MaxFromTeamRule(max_per_team=3)])

print("\nTop 5 midfielders under £8.0 with at least 500 minutes:\n")
recs = recommend_players(
    player_objects,
    limit=5,
    position="MID",
    max_price=8.0,
    min_minutes=500
)

for p, score in recs:
    print(f"- {p.name:15} | Score: {score:8.2f} | Price: {p.price}")


Top 5 midfielders under £8.0 with at least 500 minutes:

- Semenyo         | Score:   449.92 | Price: 8.0
- Rice            | Score:   426.25 | Price: 7.6
- Wilson          | Score:   416.78 | Price: 5.9
- Anderson        | Score:   409.35 | Price: 5.4
- Stach           | Score:   404.34 | Price: 4.7
